In [17]:
# import oss
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import dateparser
# from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
import requests

In [18]:
import requests

# Set your API keys
SERPAPI_KEY = "32c9a0f6319c0042a0f66eff091b1399f16c28f3daff2224083495deb08905ba"
# saumya_key = "163d0a2c7a171e2b5e1651decea27e14f37f51d274d5e83e3e89c50cccd59825"
# GROQ_API_KEY = "your_groq_api_key"

def web_search(query: str, num_results=5) -> list:
    """Search Google for relevant links using SerpAPI."""
    api_key = SERPAPI_KEY
    url = "https://serpapi.com/search"
    
    params = {
        "q": query,
        "api_key": api_key,
        "num": num_results
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extract top links
    links = [result["link"] for result in data.get("organic_results", [])[:num_results]]
    
    return links if links else ["No relevant links found."]

# Example Query
query = "sugarcane agriculture from government of india yogna"
related_links = web_search(query)
print("🔗 Related Links:", related_links)


🔗 Related Links: ['https://dfpd.gov.in/sugar_policy.html/en', 'https://agriharyana.gov.in/NFMSSugarCane', 'https://sugarcane.dac.gov.in/', 'https://dfpd.gov.in/directorate-of-sugar1.html/en', 'https://www.nfsm.gov.in/BriefNote/BN_Sugarcane.pdf']


In [5]:
llm = ChatGroq(
    api_key="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",
    model="llama-3.3-70b-versatile",
    # model="deepseek-r1-distill-llama-70b",
    # model="llama-3.3-70b-specdec",
)

In [6]:
rag_template = """
You are a ChatBot which Only have a main task to Understant the user problem and help find realated links and articals:
This is the bellow code for it 

'''python
def web_search(query: str, num_results=5) -> list:
    Search Google for relevant links using SerpAPI.
    api_key = SERPAPI_KEY
    url = "https://serpapi.com/search"
    
    params = {{
        "q": query,
        "api_key": api_key,
        "num": num_results
    }}
    
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extract top links
    links = [result["link"] for result in data.get("organic_results", [])[:num_results]]
    
    return links if links else ["No relevant links found."]
'''

What you need to do is 3 things if in the given context there is any problem and it solution
you need to find best query which i can use in the function to get the links from youtube 
and if there is any product mentioned in it context which will help solving that problem and will be used in the solution you will give the best query to find the products link on amazon and flipkart
and you will give one more query always to find the related government yojan/schemes related to that context.

And you will always give the message in JSON format no matter what always give in JSON and not think more 

Example: 
{{
    "YouTube" : "query",
    "Product" : "query",
    "Government Schemes "query"
}}

Context: {context}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": RunnablePassthrough(),}
    | rag_prompt
    | llm
    | StrOutputParser()
)


In [11]:
import json
import re
def parse_response(response):
    # Extract JSON content between {}
    match = re.search(r'\{(.|\n)*\}', response)
    if match:
        response = match.group(0)  # Extracted JSON as string
    else:
        return response
    
    if response.startswith("{"):
        response = json.loads(response)

        youtube_link = []
        product_link = []
        schemes_link = []

        if response['YouTube']:
            youtube_link = web_search(response['YouTube'])
        if response['Product']:
            product_link = web_search(response['Product'])
        if response['Government Schemes']:
            schemes_link = web_search(response['Government Schemes'])

        return youtube_link, product_link, schemes_link
    

In [14]:
user_input = input()
response = rag_chain.invoke({"context": user_input})
response = parse_response(response)

In [ ]:
print(response)

['https://eos.com/blog/how-to-grow-sugar-cane/', 'https://www.britannica.com/plant/sugarcane', 'https://www.lsuagcenter.com/nr/rdonlyres/807e6478-e556-44a8-8440-16a2cc4bfd4c/94894/sugarcaneproductionhandbook.pdf', 'https://www.ars.usda.gov/oc/utm/the-role-of-precision-agriculture-in-sugarcane-production/']
